In [1]:
# Misc imports
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")
import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
import yaml
from pathlib import Path
# Local imports
from ese.scripts.utils import get_option_product
from ese.experiment.analysis.run_inference import get_cal_stats
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'fillNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'isNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'notNA' for type <class 'pandas.core.series.Series'> is overridi

In [2]:
%%yaml inference_config

log:
    root: '?'
    gether_inference_stats: True
    summary_compute_global_metrics: True
    min_fg_pixels: 0 
    log_interval: 20 
    log_image_stats: True 
    log_pixel_stats: True 
    track_ensemble_member_scores: False 

experiment:
    exp_root: '?'
    seed: 42

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

model:
    calibrator: '?'
    normalize: False

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3
    square_diff: False 

global_calibration:
    num_prob_bins: 15
    num_classes: '?' 
    neighborhood_width: 3
    square_diff: False 
    loss_weights: None 

<IPython.core.display.Javascript object>

In [4]:
%%yaml standard_model_cfg 

# For standard datasets
#####################################
data:
    splits: ('val', 'cal')
    preload: False 

ensemble:
    combine_fn: '?'
    combine_quantity: '?'
    member_temps: None
    member_w_metric: None
    member_paths: None
    normalize: False

model:
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'max-val-dice_score' 
    pretrained_select_metric: "val-dice_score"

<IPython.core.display.Javascript object>

In [5]:
%%yaml incontext_model_cfg 

# # For incontext datasets
####################################
data:
    splits: ('val',)
    preload: False 

ensemble:
    combine_fn: '?'
    combine_quantity: '?'
    normalize: False

model:
    _type: "incontext"
    _attr:
        model: 'universeg.experiment.models.QCUNet3'
        data_support_size: 64
        mix_filters: (64, 64, 64, 64)
    pretrained_exp_root : None
    checkpoint: "max-val_od-dice_score"

<IPython.core.display.Javascript object>

In [6]:
# exp_name = '04_05_24_RandomShapes_Evaluation'
# group_dict = {
#     "dataset": "Shapes",
#     "exp_group": exp_name,
#     "model_type": "standard",
#     "base_models_group": "/storage/vbutoi/scratch/ESE/training/03_26_24_RandomCircles_MassiveEnsemble",
#     "calibrated_models_group": "/storage/vbutoi/scratch/ESE/calibration/03_26_24_RandomCircles_MassiveEnsemble_CE_Calibrated",
#     "scratch_root": "/storage/vbutoi/scratch/ESE"
# }

exp_name = '04_09_24_WBC_UniverSeg_Inference'
group_dict = {
    "dataset": "WBC",
    "exp_group": exp_name,
    "model_type": "incontext",
    "base_models_group": "/storage/vbutoi/scratch/UniverSeg/training/QC3Net_SupportSetAblation",
    "scratch_root": "/storage/vbutoi/scratch/ESE"
}

In [7]:
%%yaml submit_cfg_args

submit_opts:
    seed: 42
    gather_sub_runs: False
    max_ensemble_samples: 1 
    ens_cfg_options:
        - ('mean', 'probs')
        - ('max', 'probs')
    num_ens_membs:
        # - 1
        # - 2
        - 4
        # - 8
        # - 16
        # - 32

<IPython.core.display.Javascript object>

In [8]:
%%yaml inference_config_opts

calibrator:
    # - Uncalibrated
    # - TempScaling
    # - LTS
    - ContextualHistogramBinning 

do_ensemble:
    - True
    # - False

<IPython.core.display.Javascript object>

### Standard Calibrators.

In [9]:
inference_func = get_cal_stats
# Get the configs for the different runs.
option_set = get_ese_inference_configs(
    group_dict=group_dict,
    inf_cfg_opts=inference_config_opts,
    base_cfg_args=submit_cfg_args
)

Set seed: 42


In [10]:
len(option_set)

2

In [11]:
# Load the inference cfg from local.
##################################################
inf_cfg_root = code_root / "ese" / "experiment" / "configs" / "inference"

##################################################
with open(inf_cfg_root / "datasets" / f"{group_dict['dataset']}.yaml", 'r') as file:
    dataset_inference_cfg = yaml.safe_load(file)
with open(inf_cfg_root / "Calibration_Metrics.yaml", 'r') as file:
    cal_metrics_cfg = yaml.safe_load(file)

##################################################
base_cfg = Config(inference_config).update([calibration_cfg, dataset_inference_cfg, cal_metrics_cfg])

##################################################
# If this is an incontext model we need to update the base_cfg with the incontext model cfg.
if group_dict['model_type'] == 'incontext':
    base_cfg = base_cfg.update([incontext_model_cfg])
elif group_dict['model_type'] == 'standard':
    base_cfg = base_cfg.update([standard_model_cfg])
else:
    raise ValueError(f"Unknown model type {group_dict['model_type']}")

# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

ValueError: Must provide a value for model.normalize

## Running Jobs

In [ ]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [ ]:
from ese.experiment.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=True,
    gpu='3',
) 

In [ ]:
# from ese.experiment.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=cfgs,
#     job_func=get_cal_stats,
#     # available_gpus=['0', '1', '2', '3'],
#     available_gpus=['0', '1', '2', '3', '4', '5', '6', '7'],
# )